# Week 2 Assignment: CIFAR-10 Autoencoder

For this week, you will create a convolutional autoencoder for the [CIFAR10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset. You are free to choose the architecture of your autoencoder provided that the output image has the same dimensions as the input image.

After training, your model should meet loss and accuracy requirements when evaluated with the test dataset. You will then download the model and upload it in the classroom for grading. 

Let's begin!

***Important:*** *This colab notebook has read-only access so you won't be able to save your changes. If you want to save your work periodically, please click `File -> Save a Copy in Drive` to create a copy in your account, then work from there.*  

## Imports

In [38]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import tensorflow_datasets as tfds

from keras.models import Sequential

## Load and prepare the dataset

The [CIFAR 10](https://www.tensorflow.org/datasets/catalog/cifar10) dataset already has train and test splits and you can use those in this exercise. Here are the general steps:

* Load the train/test split from TFDS. Set `as_supervised` to `True` so it will be convenient to use the preprocessing function we provided.
* Normalize the pixel values to the range [0,1], then return `image, image` pairs for training instead of `image, label`. This is because you will check if the output image is successfully regenerated after going through your autoencoder.
* Shuffle and batch the train set. Batch the test set (no need to shuffle).


In [39]:
# preprocessing function
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0

  return image, image # dataset label is not used. replaced with the same image input.


def add_noise(image, label):
  image = tf.cast(image, tf.float32)
  noise_factor = 0.5

  noisy_image = noise_factor * tf.random.normal(image.shape)
  noisy_image = tf.clip_by_value(noisy_image, 0, 1)
  return noisy_image, image

# parameters
BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024


### START CODE HERE (Replace instances of `None` with your code) ###

# use tfds.load() to fetch the 'train' split of CIFAR-10
train_dataset = tfds.load("mnist",as_supervised=True, split="train")

# preprocess the dataset with the `map_image()` function above
train_dataset = train_dataset.map(map_image)
#train_dataset = train_dataset.map(add_noise)

# shuffle and batch the dataset
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)


# use tfds.load() to fetch the 'test' split of CIFAR-10
test_dataset = tfds.load("mnist",as_supervised=True, split="test")

# preprocess the dataset with the `map_image()` function above
test_dataset = test_dataset.map(map_image)
#test_dataset = test_dataset.map(add_noise)
# batch the dataset
test_dataset = test_dataset.batch(BATCH_SIZE)

### END CODE HERE ###

In [40]:
train_dataset.take(1)

<TakeDataset shapes: ((None, 28, 28, 1), (None, 28, 28, 1)), types: (tf.float32, tf.float32)>

## Build the Model

Create the autoencoder model. As shown in the lectures, you will want to downsample the image in the encoder layers then upsample it in the decoder path. Note that the output layer should be the same dimensions as the original image. Your input images will have the shape `(32, 32, 3)`. If you deviate from this, your model may not be recognized by the grader and may fail. 

We included a few hints to use the Sequential API below but feel free to remove it and use the Functional API just like in the ungraded labs if you're more comfortable with it. Another reason to use the latter is if you want to visualize the encoder output. As shown in the ungraded labs, it will be easier to indicate multiple outputs with the Functional API. That is not required for this assignment though so you can just stack layers sequentially if you want a simpler solution.

In [41]:
# suggested layers to use. feel free to add or remove as you see fit.
from keras.layers import Conv2D, UpSampling2D, MaxPool2D

default_pool1 = (2,2)
default_kernel1 = (3,3)
def encoder(inputs):

  conv2d_1 = Conv2D(filters=64, activation=tf.nn.relu, padding="SAME", kernel_size=default_kernel1)(inputs)
  maxPool_1 = MaxPool2D(pool_size=default_pool1)(conv2d_1)

  conv2d_2 = Conv2D(filters= 128, activation=tf.nn.relu, padding="SAME", kernel_size=default_kernel1)(maxPool_1)
  maxPool_2 = MaxPool2D(pool_size=default_pool1)(conv2d_2)

  conv2d_3 = Conv2D(filters= 256, activation=tf.nn.relu, padding="SAME", kernel_size=default_kernel1)(maxPool_2)

  return conv2d_3

def bottle_neck(inputs):
   conv2d_1 = Conv2D(filters=512, activation=tf.nn.relu, padding="SAME", kernel_size=default_kernel1)(inputs)

   return conv2d_1

def decoder(inputs):

  conv2d_1 = Conv2D(filters=256, activation=tf.nn.relu, padding="SAME", kernel_size=default_kernel1)(inputs)
  up_sample_1 = tf.keras.layers.UpSampling2D(size=(2,2))(conv2d_1)

  conv2d_2 = Conv2D(filters= 128, activation=tf.nn.relu, padding="SAME", kernel_size=default_kernel1)(up_sample_1)
  up_sample_2 = tf.keras.layers.UpSampling2D(size=(2,2))(conv2d_2)

  conv2d_3 = Conv2D(filters= 1, activation=tf.nn.relu, padding="SAME", kernel_size=default_kernel1)(up_sample_2)


  return conv2d_3

def convolutional_auto_encoder():
  '''Builds the entire autoencoder model.'''
  inputs = tf.keras.layers.Input(shape=(28, 28, 1,))
  encoder_output = encoder(inputs)
  bottleneck_output = bottle_neck(encoder_output)
  decoder_output = decoder(bottleneck_output)
  
  model = tf.keras.Model(inputs =inputs, outputs=decoder_output)
  encoder_model = tf.keras.Model(inputs=inputs, outputs=encoder_output)
  return model, encoder_model

### END CODE HERE ###
model,encoder_model = convolutional_auto_encoder()
model.summary()

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 28, 28, 64)        640       
_________________________________________________________________
max_pooling2d_45 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 7, 7, 256)         295168    
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 7, 7, 512)         118

## Configure training parameters

We have already provided the optimizer, metrics, and loss in the code below.

In [42]:
# Please do not change the model.compile() parameters
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), metrics=['accuracy'], loss='mean_squared_error')

## Training

You can now use [model.fit()](https://keras.io/api/models/model_training_apis/#fit-method) to train your model. You will pass in the `train_dataset` and you are free to configure the other parameters. As with any training, you should see the loss generally going down and the accuracy going up with each epoch. If not, please revisit the previous sections to find possible bugs.

*Note: If you get a `dataset length is infinite` error. Please check how you defined `train_dataset`. You might have included a [method that repeats the dataset indefinitely](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#repeat).*

In [43]:
# parameters (feel free to change this)
train_steps = len(train_dataset) // BATCH_SIZE 
val_steps = len(test_dataset) // BATCH_SIZE

### START CODE HERE ###
model.fit(train_dataset, steps_per_epoch=train_steps, validation_data=test_dataset, validation_steps=val_steps, epochs=40)
### END CODE HERE ###

Epoch 1/40
3/3 [==============================] - 1s 45ms/step - loss: 0.1710 - accuracy: 0.6921
Epoch 2/40
3/3 [==============================] - 0s 44ms/step - loss: 0.1086 - accuracy: 0.8113
Epoch 3/40
3/3 [==============================] - 0s 42ms/step - loss: 0.1041 - accuracy: 0.8111
Epoch 4/40
3/3 [==============================] - 0s 45ms/step - loss: 0.0866 - accuracy: 0.8067
Epoch 5/40
3/3 [==============================] - 0s 42ms/step - loss: 0.0826 - accuracy: 0.7927
Epoch 6/40
3/3 [==============================] - 0s 41ms/step - loss: 0.0757 - accuracy: 0.8133
Epoch 7/40
3/3 [==============================] - 0s 44ms/step - loss: 0.0716 - accuracy: 0.8094
Epoch 8/40
3/3 [==============================] - 0s 39ms/step - loss: 0.0667 - accuracy: 0.8027
Epoch 9/40
3/3 [==============================] - 0s 38ms/step - loss: 0.0603 - accuracy: 0.7984
Epoch 10/40
3/3 [==============================] - 0s 41ms/step - loss: 0.0565 - accuracy: 0.7860
Epoch 11/40
3/3 [============

## Model evaluation

You can use this code to test your model locally before uploading to the grader. To pass, your model needs to satisfy these two requirements:

* loss must be less than 0.01 
* accuracy must be greater than 0.6

In [44]:
result = model.evaluate(test_dataset, steps=10)

10/10 [==============================] - 0s 13ms/step - loss: 0.0085 - accuracy: 0.8105


If you did some visualization like in the ungraded labs, then you might see something like the gallery below. This part is not required.

<img src="https://drive.google.com/uc?export=view&id=12Fy-guiP-3tTPfc9IV2nHhqLvs7LwRo6" width="75%" height="75%"/>

## Save your model

Once you are satisfied with the results, you can now save your model. Please download it from the Files window on the left and go back to the Submission portal in Coursera for grading.

In [45]:
from google.colab import drive;drive.mount("/content/drive")

KeyboardInterrupt: ignored

In [46]:
model.save('mymodel.h5')

**Congratulations on completing this week's assignment!**